In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.book import *
from nltk.corpus import stopwords
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
import lightgbm as lgb
from sklearn.preprocessing import label_binarize
from sklearn.tree import DecisionTreeRegressor
from tqdm import tqdm_notebook
import joblib
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from collections import defaultdict
from gensim import corpora
from sklearn.model_selection import cross_val_score
from gensim import models
from scipy.stats import randint
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv("../4-消融实验/基本模型 + LabelEncoder(City) + 自带类别功能(Other) + Normalization/data_stand_lgb.csv")
category_col = ['attacktype1_txt', 'targsubtype1_txt', 'weapsubtype1_txt']
data[category_col] = data[category_col].astype('category')
X = data.drop(columns=['risk'], axis=1)
y = data['risk']
# kf = KFold(n_splits=10, shuffle=True, random_state=0)
# splits = kf.split(X, y)
# next(iter(splits)
data.shape

(72550, 8)

In [4]:
#lightGbm
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV

lgb_roc_scores = []
lgb_acc_scores = []
lgb_f1_scores = []
lgb_recall_scores = []
lgb_precision_scores = []
lgb_feature_importances = pd.DataFrame(index=None)
lgb_feature_importances['features'] = data.drop(['risk'], axis=1).columns
distributions = {
    'learning_rate': np.arange(0.1, 1, step=0.1),
    'min_child_samples': np.arange(5, 50, step=5),
    'max_depth':  np.arange(1, 15, step=1),
    'num_leaves': np.arange(5, 50, step=5),
    'colsample_bytree':  np.arange(0.1, 1, step=0.1),
    'reg_alpha': randint(0, 100),
    'reg_lambda': randint(0, 100),
}

# print("第 %d 折\n" % (k + 1))
# X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
# y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]

# labels = [0, 1, 2, 3]
# y_one_hot = label_binarize(y_test, classes=labels)

LGB = lgb.LGBMClassifier(random_state=0, categorical_feature=category_col)
clf =  RandomizedSearchCV(LGB, distributions, random_state=0, scoring='recall_weighted', cv=10)
search = clf.fit(X, y)
print('最佳超参数是：' + str(search.best_params_))
print('最好成绩是：' + str(search.best_score_))

最佳超参数是：{'colsample_bytree': 0.5, 'learning_rate': 0.8, 'max_depth': 4, 'min_child_samples': 15, 'num_leaves': 40, 'reg_alpha': 28, 'reg_lambda': 34}
最好成绩是：0.6417643004824258


In [5]:
data = pd.read_csv("../4-消融实验/基本模型 + LabelEncoder(City) + 自带类别功能(Other) + Normalization/data_stand_lgb.csv")
category_col = ['attacktype1_txt', 'targsubtype1_txt', 'weapsubtype1_txt']
data[category_col] = data[category_col].astype('category')
X = data.drop(columns=['risk'], axis=1)
y = data['risk']
kf = KFold(n_splits=10, shuffle=True, random_state=0)
splits = kf.split(X, y)
next(iter(splits))

(array([    0,     1,     2, ..., 72547, 72548, 72549]),
 array([    3,    14,    18, ..., 72525, 72542, 72544]))

In [6]:
#lightGbm
import lightgbm as lgb

lgb_roc_scores = []
lgb_acc_scores = []
lgb_f1_scores = []
lgb_recall_scores = []
lgb_precision_scores = []
lgb_feature_importances = pd.DataFrame(index=None)
lgb_feature_importances['features'] = data.drop(['risk'], axis=1).columns

for k, (train_indices, test_indices) in enumerate(splits):
    print("第 %d 折\n" % (k + 1))
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]
    
    labels = [0, 1, 2, 3]
    y_one_hot = label_binarize(y_test, classes=labels)
    
    LGB = lgb.LGBMClassifier(random_state=0, 
                             learning_rate = search.best_params_['learning_rate'],
                             min_child_samples = search.best_params_['min_child_samples'],
                             max_depth = search.best_params_['max_depth'],
                             num_leaves = search.best_params_['num_leaves'],
                             colsample_bytree = search.best_params_['colsample_bytree'],
                             reg_alpha = search.best_params_['reg_alpha'],
                             reg_lambda = search.best_params_['reg_lambda'],
                            )
    LGB.fit(X_train, y_train)
    lgb_feature_importances[f'fold_{k+1}'] = LGB.feature_importances_
    y_pred_prob = LGB.predict_proba(X_test)
    y_pred = LGB.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_one_hot, y_pred_prob, multi_class="ovo", average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
#     G_mean = math.sqrt(recall * specificity)
    print(f" Fold {k + 1} | " )
    print(f" AUC_ROC: { roc_auc * 100}%" )
    print(f" ACC: { acc * 100}%" )
    print(f" F1: { f1 * 100}%" )
    print(f" RECALL: { recall * 100}%" )
    print(f" PRECISION: { precision * 100}%" )
    lgb_f1_scores.append(f1)
    lgb_roc_scores.append(roc_auc)
    lgb_acc_scores.append(acc)
    lgb_recall_scores.append(recall)
    lgb_precision_scores.append(precision)
    
print(f'average roc score: {np.mean(lgb_roc_scores)}')
print(f'average acc_score: {np.mean(lgb_acc_scores)}')
print(f'average f1_score: {np.mean(lgb_f1_scores)}')
print(f'average recall_score: {np.mean(lgb_recall_scores)}')
print(f'average precision_score: {np.mean(lgb_precision_scores)}')

第 1 折

 Fold 1 | 
 AUC_ROC: 85.95816765413711%
 ACC: 65.66505858028945%
 F1: 65.14316064664533%
 RECALL: 65.66505858028945%
 PRECISION: 65.13349828159232%
第 2 折

 Fold 2 | 
 AUC_ROC: 86.23802796130717%
 ACC: 65.87181254307374%
 F1: 65.28208742669756%
 RECALL: 65.87181254307374%
 PRECISION: 65.28940973659638%
第 3 折

 Fold 3 | 
 AUC_ROC: 86.53705736262177%
 ACC: 66.42315644383184%
 F1: 65.90760005569757%
 RECALL: 66.42315644383184%
 PRECISION: 65.95040783054546%
第 4 折

 Fold 4 | 
 AUC_ROC: 86.69413911918066%
 ACC: 66.31288766368021%
 F1: 65.74880608297073%
 RECALL: 66.31288766368021%
 PRECISION: 65.6638070715895%
第 5 折

 Fold 5 | 
 AUC_ROC: 86.11219478450145%
 ACC: 66.05099931082012%
 F1: 65.60341457388196%
 RECALL: 66.05099931082012%
 PRECISION: 65.54968754295423%
第 6 折

 Fold 6 | 
 AUC_ROC: 86.57984589875466%
 ACC: 66.35423845623708%
 F1: 65.96981264665878%
 RECALL: 66.35423845623708%
 PRECISION: 65.99613609483886%
第 7 折

 Fold 7 | 
 AUC_ROC: 86.26581556545277%
 ACC: 66.21640248104755%

In [7]:
data = pd.read_csv("../4-消融实验/基本模型 + LabelEncoder(City) + 自带类别功能(Other) + Normalization/data_stand_lgb.csv")
category_col = ['attacktype1_txt', 'targsubtype1_txt', 'weapsubtype1_txt']
data[category_col] = data[category_col].astype('category')
X = data.drop(columns=['risk'], axis=1)
y = data['risk']

In [8]:
import hyperopt


def hyperopt_objective(params):
    
    model = lgb.LGBMClassifier(
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
#         n_estimators=int(params['n_estimators']),
#         subsample=params["subsample"],
        colsample_bytree=params["colsample_bytree"],
#         min_data_in_leaf=int(params["min_data_in_leaf"])  # 最小叶子节点样本数
        min_child_samples=params["min_child_samples"],
        num_leaves=params["num_leaves"],
        reg_alpha=params["reg_alpha"],
        reg_lambda=params["reg_lambda"],
    )
    
    res = np.mean(cross_val_score(model, X, y, cv=10, scoring='recall_weighted'))
    print("*" * 30)
    print(params)
    print("roc_auc: {}".format(res))
    
    return 1 - res

# 定义调参空间
params_space = {
    'max_depth': hyperopt.hp.randint('max_depth', 15),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
#     'n_estimators': hyperopt.hp.quniform("n_estimators", 100, 200, 20),
#     'subsample': hyperopt.hp.uniform('subsample', 0.6, 1),
    'colsample_bytree': hyperopt.hp.uniform('colsample_bytree', 0.6, 1),
#     'min_data_in_leaf': hyperopt.hp.quniform('min_data_in_leaf', 3, 15, 2)
    'min_child_samples': hyperopt.hp.randint('min_child_samples', 50),
    'num_leaves': hyperopt.hp.randint('num_leaves', 100),
    'reg_alpha': hyperopt.hp.randint('reg_alpha', 100),
    'reg_lambda': hyperopt.hp.randint('reg_lambda', 100),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=20,
    trials=trials)

print("最佳参数")
print(best)

******************************                        
{'colsample_bytree': 0.7812685028644614, 'learning_rate': 0.18126626869612242, 'max_depth': 9, 'min_child_samples': 27, 'num_leaves': 22, 'reg_alpha': 96, 'reg_lambda': 43}
roc_auc: 0.6354927636113026                           
******************************                                                  
{'colsample_bytree': 0.8694341584586537, 'learning_rate': 0.21385341046261128, 'max_depth': 2, 'min_child_samples': 30, 'num_leaves': 74, 'reg_alpha': 86, 'reg_lambda': 52}
roc_auc: 0.6270572019297036                                                     
******************************                                                  
{'colsample_bytree': 0.8641763917450669, 'learning_rate': 0.4733699401163345, 'max_depth': 2, 'min_child_samples': 12, 'num_leaves': 84, 'reg_alpha': 84, 'reg_lambda': 31}
roc_auc: 0.6291660923501033                                                     
******************************                  

In [9]:
print(best['learning_rate'])

0.27852359632211016


In [10]:
data = pd.read_csv("../4-消融实验/基本模型 + LabelEncoder(City) + 自带类别功能(Other) + Normalization/data_stand_lgb.csv")
category_col = ['attacktype1_txt', 'targsubtype1_txt', 'weapsubtype1_txt']
data[category_col] = data[category_col].astype('category')
X = data.drop(columns=['risk'], axis=1)
y = data['risk']
kf = KFold(n_splits=10, shuffle=True, random_state=0)
splits = kf.split(X, y)
next(iter(splits))

(array([    0,     1,     2, ..., 72547, 72548, 72549]),
 array([    3,    14,    18, ..., 72525, 72542, 72544]))

In [11]:
#lightGbm
import lightgbm as lgb

lgb_roc_scores = []
lgb_acc_scores = []
lgb_f1_scores = []
lgb_recall_scores = []
lgb_precision_scores = []
lgb_feature_importances = pd.DataFrame(index=None)
lgb_feature_importances['features'] = data.drop(['risk'], axis=1).columns

for k, (train_indices, test_indices) in enumerate(splits):
    print("第 %d 折\n" % (k + 1))
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]
    
    labels = [0, 1, 2, 3]
    y_one_hot = label_binarize(y_test, classes=labels)
    
    LGB = lgb.LGBMClassifier(random_state=0, 
                             learning_rate = best['learning_rate'],
                             min_child_samples = best['min_child_samples'],
                             max_depth = best['max_depth'],
                             num_leaves = best['num_leaves'],
                             colsample_bytree = best['colsample_bytree'],
                             reg_alpha = best['reg_alpha'],
                             reg_lambda = best['reg_lambda'],
                            )
    LGB.fit(X_train, y_train)
    lgb_feature_importances[f'fold_{k+1}'] = LGB.feature_importances_
    y_pred_prob = LGB.predict_proba(X_test)
    y_pred = LGB.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_one_hot, y_pred_prob, multi_class="ovo", average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
#     G_mean = math.sqrt(recall * specificity)
    print(f" Fold {k + 1} | " )
    print(f" AUC_ROC: { roc_auc * 100}%" )
    print(f" ACC: { acc * 100}%" )
    print(f" F1: { f1 * 100}%" )
    print(f" RECALL: { recall * 100}%" )
    print(f" PRECISION: { precision * 100}%" )
    lgb_f1_scores.append(f1)
    lgb_roc_scores.append(roc_auc)
    lgb_acc_scores.append(acc)
    lgb_recall_scores.append(recall)
    lgb_precision_scores.append(precision)
    
print(f'average roc score: {np.mean(lgb_roc_scores)}')
print(f'average acc_score: {np.mean(lgb_acc_scores)}')
print(f'average f1_score: {np.mean(lgb_f1_scores)}')
print(f'average recall_score: {np.mean(lgb_recall_scores)}')
print(f'average precision_score: {np.mean(lgb_precision_scores)}')

第 1 折

 Fold 1 | 
 AUC_ROC: 86.51603137014942%
 ACC: 66.64369400413509%
 F1: 66.1868718210684%
 RECALL: 66.64369400413509%
 PRECISION: 66.1692665923029%
第 2 折

 Fold 2 | 
 AUC_ROC: 86.80144684779822%
 ACC: 66.42315644383184%
 F1: 65.90740396537275%
 RECALL: 66.42315644383184%
 PRECISION: 65.99052019630396%
第 3 折

 Fold 3 | 
 AUC_ROC: 86.9820972700717%
 ACC: 66.7539627842867%
 F1: 66.24732663374458%
 RECALL: 66.7539627842867%
 PRECISION: 66.2760082611214%
第 4 折

 Fold 4 | 
 AUC_ROC: 87.19733338021747%
 ACC: 67.26395589248794%
 F1: 66.76196767602312%
 RECALL: 67.26395589248794%
 PRECISION: 66.68204750483044%
第 5 折

 Fold 5 | 
 AUC_ROC: 86.67551132337691%
 ACC: 66.68504479669194%
 F1: 66.22338421707548%
 RECALL: 66.68504479669194%
 PRECISION: 66.22416625403079%
第 6 折

 Fold 6 | 
 AUC_ROC: 86.99531664665263%
 ACC: 67.13990351481736%
 F1: 66.78923674644798%
 RECALL: 67.13990351481736%
 PRECISION: 66.84574332901887%
第 7 折

 Fold 7 | 
 AUC_ROC: 86.6557998491788%
 ACC: 66.79531357684355%
 F1: 

In [12]:
data = pd.read_csv("../4-消融实验/基本模型 + LabelEncoder(City) + 自带类别功能(Other) + Normalization/data_stand_lgb.csv")
category_col = ['attacktype1_txt', 'targsubtype1_txt', 'weapsubtype1_txt']
data[category_col] = data[category_col].astype('category')
X = data.drop(columns=['risk'], axis=1)
y = data['risk']
kf = KFold(n_splits=10, shuffle=True, random_state=0)
splits = kf.split(X, y)
next(iter(splits))

(array([    0,     1,     2, ..., 72547, 72548, 72549]),
 array([    3,    14,    18, ..., 72525, 72542, 72544]))

In [13]:
#lightGbm
import lightgbm as lgb

lgb_roc_scores = []
lgb_acc_scores = []
lgb_f1_scores = []
lgb_recall_scores = []
lgb_precision_scores = []
lgb_feature_importances = pd.DataFrame(index=None)
lgb_feature_importances['features'] = data.drop(['risk'], axis=1).columns

for k, (train_indices, test_indices) in enumerate(splits):
    print("第 %d 折\n" % (k + 1))
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]
    
    labels = [0, 1, 2, 3]
    y_one_hot = label_binarize(y_test, classes=labels)
    
    LGB = lgb.LGBMClassifier(random_state=0, 
                             learning_rate = 0.1,
                             min_child_samples = 10,
                             max_depth = 2,
                             num_leaves = 20,
                             colsample_bytree = 0.2,
                             reg_alpha = 0,
                             reg_lambda = 0,                                                                      
                            )
    LGB.fit(X_train, y_train)
    lgb_feature_importances[f'fold_{k+1}'] = LGB.feature_importances_
    y_pred_prob = LGB.predict_proba(X_test)
    y_pred = LGB.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_one_hot, y_pred_prob, multi_class="ovo", average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
#     G_mean = math.sqrt(recall * specificity)
    print(f" Fold {k + 1} | " )
    print(f" AUC_ROC: { roc_auc * 100}%" )
    print(f" ACC: { acc * 100}%" )
    print(f" F1: { f1 * 100}%" )
    print(f" RECALL: { recall * 100}%" )
    print(f" PRECISION: { precision * 100}%" )
    lgb_f1_scores.append(f1)
    lgb_roc_scores.append(roc_auc)
    lgb_acc_scores.append(acc)
    lgb_recall_scores.append(recall)
    lgb_precision_scores.append(precision)
    
print(f'average roc score: {np.mean(lgb_roc_scores)}')
print(f'average acc_score: {np.mean(lgb_acc_scores)}')
print(f'average f1_score: {np.mean(lgb_f1_scores)}')
print(f'average recall_score: {np.mean(lgb_recall_scores)}')
print(f'average precision_score: {np.mean(lgb_precision_scores)}')

第 1 折

 Fold 1 | 
 AUC_ROC: 79.81779959339408%
 ACC: 57.10544452101999%
 F1: 55.79277789140823%
 RECALL: 57.10544452101999%
 PRECISION: 56.43366141149466%
第 2 折

 Fold 2 | 
 AUC_ROC: 80.15689672836037%
 ACC: 57.780840799448654%
 F1: 56.322940082120866%
 RECALL: 57.780840799448654%
 PRECISION: 57.038592507484346%
第 3 折

 Fold 3 | 
 AUC_ROC: 80.40845129936302%
 ACC: 57.56030323914542%
 F1: 55.9879791542063%
 RECALL: 57.56030323914542%
 PRECISION: 56.819609844377915%
第 4 折

 Fold 4 | 
 AUC_ROC: 81.15685873840998%
 ACC: 59.09028256374914%
 F1: 57.26279420971121%
 RECALL: 59.09028256374914%
 PRECISION: 57.75781676178758%
第 5 折

 Fold 5 | 
 AUC_ROC: 80.30158138836183%
 ACC: 58.63542384562371%
 F1: 57.16049130091345%
 RECALL: 58.63542384562371%
 PRECISION: 57.81246409354458%
第 6 折

 Fold 6 | 
 AUC_ROC: 80.57276996706756%
 ACC: 57.94624396967608%
 F1: 56.7916768063418%
 RECALL: 57.94624396967608%
 PRECISION: 57.446112339825085%
第 7 折

 Fold 7 | 
 AUC_ROC: 80.04063717206276%
 ACC: 57.4638180565

In [14]:
data = pd.read_csv("../4-消融实验/基本模型 + LabelEncoder(City) + 自带类别功能(Other) + Normalization/data_stand_lgb.csv")
category_col = ['attacktype1_txt', 'targsubtype1_txt', 'weapsubtype1_txt']
data[category_col] = data[category_col].astype('category')
X = data.drop(columns=['risk'], axis=1)
y = data['risk']
kf = KFold(n_splits=10, shuffle=True, random_state=0)
splits = kf.split(X, y)
next(iter(splits))

(array([    0,     1,     2, ..., 72547, 72548, 72549]),
 array([    3,    14,    18, ..., 72525, 72542, 72544]))

In [15]:
#lightGbm
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

lgb_roc_scores = []
lgb_acc_scores = []
lgb_f1_scores = []
lgb_recall_scores = []
lgb_precision_scores = []
lgb_feature_importances = pd.DataFrame(index=None)
lgb_feature_importances['features'] = data.drop(['risk'], axis=1).columns
distributions = {
    'learning_rate': np.arange(0.1, 1, step=0.5),
    'min_child_samples': np.arange(10, 50, step=20),
    'max_depth':  np.arange(5, 15, step=5),
    'num_leaves': np.arange(10, 50, step=20),
    'colsample_bytree':  np.arange(0.1, 1, step=0.5),
    'reg_alpha':  np.arange(0, 100, step=50),
    'reg_lambda':  np.arange(0, 100, step=50),
}

LGB = lgb.LGBMClassifier(random_state=0, categorical_feature=category_col)
clf =  GridSearchCV(LGB, distributions, scoring='recall_weighted', cv=10)
search = clf.fit(X, y)
print('最佳超参数是：' + str(search.best_params_))
print('最好成绩是：' + str(search.best_score_))

最佳超参数是：{'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 10, 'min_child_samples': 10, 'num_leaves': 30, 'reg_alpha': 0, 'reg_lambda': 0}
最好成绩是：0.6471950379048932


In [16]:
#lightGbm
import lightgbm as lgb

lgb_roc_scores = []
lgb_acc_scores = []
lgb_f1_scores = []
lgb_recall_scores = []
lgb_precision_scores = []
lgb_feature_importances = pd.DataFrame(index=None)
lgb_feature_importances['features'] = data.drop(['risk'], axis=1).columns

for k, (train_indices, test_indices) in enumerate(splits):
    print("第 %d 折\n" % (k + 1))
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]
    
    labels = [0, 1, 2, 3]
    y_one_hot = label_binarize(y_test, classes=labels)
    
    LGB = lgb.LGBMClassifier(random_state=0, 
                             learning_rate = search.best_params_['learning_rate'],
                             min_child_samples = search.best_params_['min_child_samples'],
                             max_depth = search.best_params_['max_depth'],
                             num_leaves = search.best_params_['num_leaves'],
                             colsample_bytree = search.best_params_['colsample_bytree'],
                             reg_alpha = search.best_params_['reg_alpha'],
                             reg_lambda = search.best_params_['reg_lambda'],                                                                      
                            )
    LGB.fit(X_train, y_train)
    lgb_feature_importances[f'fold_{k+1}'] = LGB.feature_importances_
    y_pred_prob = LGB.predict_proba(X_test)
    y_pred = LGB.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_one_hot, y_pred_prob, multi_class="ovo", average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
#     G_mean = math.sqrt(recall * specificity)
    print(f" Fold {k + 1} | " )
    print(f" AUC_ROC: { roc_auc * 100}%" )
    print(f" ACC: { acc * 100}%" )
    print(f" F1: { f1 * 100}%" )
    print(f" RECALL: { recall * 100}%" )
    print(f" PRECISION: { precision * 100}%" )
    lgb_f1_scores.append(f1)
    lgb_roc_scores.append(roc_auc)
    lgb_acc_scores.append(acc)
    lgb_recall_scores.append(recall)
    lgb_precision_scores.append(precision)
    
print(f'average roc score: {np.mean(lgb_roc_scores)}')
print(f'average acc_score: {np.mean(lgb_acc_scores)}')
print(f'average f1_score: {np.mean(lgb_f1_scores)}')
print(f'average recall_score: {np.mean(lgb_recall_scores)}')
print(f'average precision_score: {np.mean(lgb_precision_scores)}')

第 1 折

 Fold 1 | 
 AUC_ROC: 86.52963773433171%
 ACC: 66.32667126119918%
 F1: 65.72997944523684%
 RECALL: 66.32667126119918%
 PRECISION: 65.79532021943508%
第 2 折

 Fold 2 | 
 AUC_ROC: 86.65167595691017%
 ACC: 66.28532046864231%
 F1: 65.69926968352769%
 RECALL: 66.28532046864231%
 PRECISION: 65.87364930300829%
第 3 折

 Fold 3 | 
 AUC_ROC: 86.92649149034669%
 ACC: 66.31288766368021%
 F1: 65.67519702453598%
 RECALL: 66.31288766368021%
 PRECISION: 65.82389373327571%
第 4 折

 Fold 4 | 
 AUC_ROC: 87.25625360076471%
 ACC: 66.85044796691938%
 F1: 66.1353341497242%
 RECALL: 66.85044796691938%
 PRECISION: 66.14339191717922%
第 5 折

 Fold 5 | 
 AUC_ROC: 86.68158756370194%
 ACC: 66.57477601654031%
 F1: 65.93081560901722%
 RECALL: 66.57477601654031%
 PRECISION: 66.07544049559228%
第 6 折

 Fold 6 | 
 AUC_ROC: 87.08654186324215%
 ACC: 67.00206753962784%
 F1: 66.49399493174359%
 RECALL: 67.00206753962784%
 PRECISION: 66.61338857094134%
第 7 折

 Fold 7 | 
 AUC_ROC: 86.49023875652455%
 ACC: 66.1474844934528%
